In [3]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
train = 'melanoma_cancer_dataset/train'

# set all images to 224x224
def resize_images(folder):
    for filename in os.listdir(folder):
        img = Image.open(folder + '\\' + filename)
        img = img.resize((224, 224))
        img.save(folder + '\\' + filename)

resize_images(train + '\\benign')
resize_images(train + '\\malignant')

In [5]:
test = 'melanoma_cancer_dataset/test'
resize_images(test + '\\benign')
resize_images(test + '\\malignant')


In [6]:
# find number of training and testing samples
num_train = 0
num_test = 0
for folder in os.listdir(train):
    num_train += len(os.listdir(train + '\\' + folder))
for folder in os.listdir(test):
    num_test += len(os.listdir(test + '\\' + folder))
    
batch_size = 32

In [7]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)




Found 9605 images belonging to 2 classes.


In [8]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
   
)


test_generator = test_datagen.flow_from_directory(
    test,  
    target_size=(224, 224),
    batch_size= batch_size,
    class_mode='binary',  #
    shuffle=False  
)

Found 1000 images belonging to 2 classes.


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),  # Helps reduce overfitting
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # 'sigmoid' for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [10]:
steps_per_epoch = math.ceil(num_train / batch_size)
validation_steps = math.ceil(num_test / batch_size)

In [11]:
# Train the model
early_stopping = EarlyStopping(
    monitor='val_loss',  # or 'val_accuracy'
    patience=3,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True,  # Restores model weights from the epoch with the best value of the monitored metric.
    verbose=1
)
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=15,
    validation_data=test_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)

Epoch 1/15


301/301 [==============================] - 365s 1s/step - loss: 0.4777 - accuracy: 0.7973 - val_loss: 0.3124 - val_accuracy: 0.8760
Epoch 2/15
301/301 [==============================] - 306s 1s/step - loss: 0.3532 - accuracy: 0.8504 - val_loss: 0.2822 - val_accuracy: 0.8920
Epoch 3/15
301/301 [==============================] - 282s 936ms/step - loss: 0.3327 - accuracy: 0.8606 - val_loss: 0.2999 - val_accuracy: 0.8640
Epoch 4/15
301/301 [==============================] - 272s 901ms/step - loss: 0.3180 - accuracy: 0.8693 - val_loss: 0.2868 - val_accuracy: 0.8910
Epoch 5/15
301/301 [==============================] - 276s 915ms/step - loss: 0.3153 - accuracy: 0.8693 - val_loss: 0.2570 - val_accuracy: 0.8930
Epoch 6/15
301/301 [==============================] - 282s 938ms/step - loss: 0.3084 - accuracy: 0.8690 - val_loss: 0.2897 - val_accuracy: 0.8900
Epoch 7/15
301/301 [==============================] - 265s 880ms/step - loss: 0.2938 - accuracy: 0.8781 - val_loss: 0.2465 - val

In [12]:
# evaluate the model

test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


31/31 [==============================] - 5s 154ms/step - loss: 0.2379 - accuracy: 0.9062
Test Loss: 0.23786503076553345, Test Accuracy: 0.90625


In [13]:
# make predictions
predictions = model.predict(test_generator, steps= validation_steps)
predicted_classes = predictions > 0.5

32/32 [==============================] - 5s 151ms/step


In [14]:
# classification report
from sklearn.metrics import classification_report
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

              precision    recall  f1-score   support

      benign       0.88      0.93      0.90       500
   malignant       0.93      0.87      0.90       500

    accuracy                           0.90      1000
   macro avg       0.90      0.90      0.90      1000
weighted avg       0.90      0.90      0.90      1000



In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_classes, predicted_classes)
# print cm with labels
print(cm)


[[467  33]
 [ 66 434]]


In [16]:
evaluate_result = model.evaluate(test_generator, steps=validation_steps)
print(f'Loss: {evaluate_result[0]}, Accuracy: {evaluate_result[1]}')

32/32 [==============================] - 5s 138ms/step - loss: 0.2458 - accuracy: 0.9010
Loss: 0.24584858119487762, Accuracy: 0.9010000228881836


In [17]:
# export model as API
model.save('melanoma_cancer_model.h5')


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
